# PA004 - HEALTH INSURANCE CROS-SELL

Last version of project, according Machine Learning Fundamentals course

## 0 - IMPORTS AND FUNCTIONS

In [1]:
import warnings
import optuna
import random
import pickle

import lightgbm                                               as lgbm
import scikitplot                                             as skplt
import matplotlib.patches                                     as mpatches
import matplotlib.pyplot                                      as plt
import numpy                                                  as np
import plotly.express                                         as px
import pandas                                                 as pd
import seaborn                                                as sns

from sklearn                 import preprocessing             as pp
from sklearn                 import model_selection           as ms
from sklearn                 import neighbors                 as nh
from sklearn                 import ensemble                  as en

from IPython.core.display    import HTML
from IPython.display         import Image
from xgboost                 import XGBClassifier
from sklearn.metrics         import log_loss
from sklearn.model_selection import StratifiedKFold
from optuna.integration      import LightGBMPruningCallback

warnings.filterwarnings( 'ignore' )

C:\Users\perot\anaconda3\envs\health_insurance_venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 0.1 - HELPER FUNCTIONS

In [2]:
def recall_at_k (data, k=20000):
    # Reset Index
    data = data.reset_index( drop=True )

    # Create Ranking Order
    data['ranking'] = data.index + 1

    data['recall_at_k'] = data['response'].cumsum() / data['response'].sum()

    return data.loc[k, 'recall_at_k'] 

def precision_at_k (data, k=20000):
    # Reset Index
    data = data.reset_index( drop=True )

    # Create Ranking Order
    data['ranking'] = data.index + 1

    data['precision_at_k'] = data['response'].cumsum() / data['ranking']

    return data.loc[k, 'precision_at_k']

def cramer_v( x, y):
    cm = pd.crosstab( x, y).values   
    n = cm.sum()
    r, k = cm.shape    
    
    chi2 = ss.chi2_contingency( cm )[0]
    chi2corr = max( 0, chi2 - (k-1)*(r-1)/(n-1) )
    kcorr = k - (k-1)**2/(n-1)
    rcorr = r - (r-1)**2/(n-1) 
    
    return np.sqrt( (chi2corr/n) / ( min( kcorr-1, rcorr-1 ) ) )

def prediction_score(model, data):
    # Separating train and validation dataset for each kfold
    x_train = data.drop(columns=['response'])
    y_train = data['response']
    
    # prediction probability (score)
    yhat_proba = model.predict_proba(x_train)[:, 1].tolist()
    
    # merging score to dataset
    data_scored = data.copy()
    data_scored['score'] = yhat_proba
    
    # sort
    data_scored = data_scored.sort_values('score', ascending=False)
    
    # precision and recall
    data_scored = data_scored.reset_index(drop=True)
    data_scored['n_samples'] = data_scored.index + 1
    data_scored['precision_at_k'] = data_scored['response'].cumsum() / data_scored['n_samples']
    data_scored['recall_at_k'] = data_scored['response'].cumsum() / data_scored['response'].sum()
    
    return data_scored

def cross_validation(kfold, modelName, model, data, at_k):
    # Number of folds
    fold=ms.StratifiedKFold(n_splits = kfold, shuffle=True, random_state=42)
    
    # Performance variables
    precision_list = []
    recall_list = []
    cv_performance = {}
    
    for train_cv,val_cv in fold.split(data, data['response']):
        
        # Separating train and validation dataset for each kfold
        # training data
        x_train_fold = data.iloc[train_cv]
        x_train_fold = x_train_fold[cols_selected]
        
        y_train_fold = data['response'].iloc[train_cv]
        
        # validation data
        x_val_fold = data.iloc[val_cv]
        x_val_fold = x_val_fold[cols_selected]
        
        y_val_fold = data['response'].iloc[val_cv]
        
        # fitting the model
        model_fitted = model.fit(x_train_fold,y_train_fold)
        
        # getting the prediction probability
        x_val_fold['response'] = y_val_fold
        val_scored = prediction_score(model_fitted, x_val_fold)
        
        # Getting precision and recall at k
        precision = val_scored.loc[at_k, 'precision_at_k']
        recall = val_scored.loc[at_k, 'recall_at_k']
        
        precision_list.append(precision)
        recall_list.append(recall)
        
    # calculating the mean and std performance of all kfolds
    precision_cv = np.round(np.mean(precision_list),4).astype(float)
    std_precision_cv = '+/-' + np.round(np.std(precision_list),4).astype(str)
    recall_cv = np.round(np.mean(recall_list),4).astype(float)
    std_recall_cv = ' +/- ' + np.round(np.std(recall_list),4).astype(str)
        
    cv_performance[modelName] = [precision_cv, std_precision_cv, recall_cv, std_recall_cv]
    model_performance_cv = pd.DataFrame(cv_performance, index=['precision_at_k', 'std_precision', 'recall_at_k', 'std_recall'])

    return model_performance_cv

def ml_performance( model_name, precision_atK, recall_atK ):
    
    return pd.DataFrame( {'Model Name': model_name,
                          'Precision_at_K': precision_atK,
                          'Recall_at_K': recall_atK}, index=[0])

def data_preparation (df_prep):
    # Fitting vehicle_age column
    df_prep['vehicle_age'] = df_prep['vehicle_age'].apply(lambda x: 1 if (x == '< 1 Year') else 2 if (x == '1-2 Year') else 3)

    # Fitting vehicle_damage column
    df_prep['vehicle_damage'] = df_prep['vehicle_damage'].apply(lambda x: 1 if (x == 'Yes') else 0)
    
    # STANDARDIZATION

    # annual premium
    df_prep.loc[:, 'annual_premium'] = ss_ap.transform( df_prep[['annual_premium']].values )



    # REESCALING

    # age
    df_prep.loc[:, 'age'] = mms_age.transform( df_prep[['age']].values )

    # vintage
    df_prep.loc[:, 'vintage'] = mms_vintage.transform( df_prep[['vintage']].values )



    # ENCODER

    # policy sales channel
    df_prep.loc[:, 'policy_sales_channel'] = df_prep['policy_sales_channel'].map( fe_policy_sales_channel )

    # region code
    df_prep.loc[:, 'region_code'] = df_prep.loc[:, 'region_code'].map( fe_region_code )

    # gender
    df_prep.loc[:, 'gender'] = df_prep.loc[:, 'gender'].map( fe_gender )

    # FILL NAN VALUES
    df_prep = df_prep.fillna(0)
    
    # Feature Selection
    cols_selected = ['vintage', 'annual_premium','age','region_code','vehicle_damage','policy_sales_channel','previously_insured','vehicle_age']
    
    return( df_prep[cols_selected] )

def jupyter_settings():
    %matplotlib inline
    %pylab inline
    
    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [18, 8]
    plt.rcParams['font.size'] = 24
    
    display( HTML( '<style>.container {width:100% !important; }</style>') )
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False )
    
    sns.set()

In [3]:
jupyter_settings()

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


## 1 - LOADING AND UNDESTANDING DATA

In [4]:
# All data - Linux
#df = pd.read_csv('/home/reng/Documents/ds_repos/Projects/Health_Insurance_Cross_Sell/data/raw/data.csv')

# All Data - Windows
df = pd.read_csv('C:/Users/perot/Documents/ds_repos/projects/Health_Insurance_Cross_Sell/data/raw/data.csv') 

### 1.1 - SPLIT DATASET INTO TRAINING, TEST AND VALIDATION

In [5]:
# Criando os datasets de treino e validacao
X = df.drop( 'response', axis=1 )
y = df['response'].copy()
x_train, x_valid, y_train, y_valid = ms.train_test_split( X, y, test_size=0.2 )

# Criando dataset de teste, a partir do dataset de treino
x_train, x_test, y_train, y_test = ms.train_test_split( x_train, y_train, test_size=0.2 )

print(f" x_train: {x_train.shape}\n y_train: {y_train.shape}\n x_valid: {x_valid.shape}\n y_valid: {y_valid.shape}\n x_test: {x_test.shape}\n y_test: {y_test.shape}\n")

 x_train: (243909, 11)
 y_train: (243909,)
 x_valid: (76222, 11)
 y_valid: (76222,)
 x_test: (60978, 11)
 y_test: (60978,)



### 1.2 - DATA DESCRIPTION

In [7]:
df1 = x_train.copy()

In [8]:
# Data Dimensions
print( 'Number of Rows: {}'.format( df1.shape[0] ) )
print( 'Numfer of Cols: {}'.format( df1.shape[1] ) )

Number of Rows: 243909
Numfer of Cols: 11


In [ ]:
# Data Types
df1.dtypes

In [ ]:
# Changing data types
df1['region_code'] = df1['region_code'].astype(object)
df1['policy_sales_channel'] = df1['policy_sales_channel'].astype(object)

In [ ]:
# Check NAN Values
df1.isna().sum()

### 1.3 - DATA DESCRIPTIVE

In [ ]:
df1.info()

In [ ]:
df1.describe().T

In [ ]:
# Select columns where the content is different 0 or 1.
num_attributes = df1[['age','annual_premium','vintage']]
cat_attributes = df1[['driving_license','region_code','policy_sales_channel','previously_insured','gender','vehicle_age','vehicle_damage']]
target_atribute = df1['response']

In [ ]:
num_attributes.hist(bins=25, figsize = (10, 5 ));

In [ ]:
# Central Tendency - mean, median
ct1 = pd.DataFrame( num_attributes.apply( np.mean ) ).T
ct2 = pd.DataFrame( num_attributes.apply( np.median ) ).T

# Dispersion - std, min, max, range, skew, kurtosis
d1 = pd.DataFrame( num_attributes.apply( np.std ) ).T
d2 = pd.DataFrame( num_attributes.apply( min ) ).T
d3 = pd.DataFrame( num_attributes.apply( max ) ).T
d4 = pd.DataFrame( num_attributes.apply( lambda x: x.max() - x.min() ) ).T
d5 = pd.DataFrame( num_attributes.apply( lambda x: x.skew() ) ).T
d6 = pd.DataFrame( num_attributes.apply( lambda x: x.kurtosis() ) ).T

# Concatenar
m = pd.concat( [ d2, d3, d4, ct1, ct2, d1, d5, d6 ] ).T.reset_index()
m.columns = ['attributes', 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis']
m

In [ ]:
# Looking at the distribuition of the following variables.
fig, axs = plt.subplots(3, figsize = (8, 10))
sns.distplot(df1['age'], ax=axs[0], bins=10)
sns.distplot(df1[df1['annual_premium']<100000]['annual_premium'], ax=axs[1])
sns.distplot(df1['vintage'], ax=axs[2]);

In [ ]:
fig, axs = plt.subplots(ncols=3, nrows=2, figsize= (15, 8))

sns.countplot(x = 'driving_license', palette = 'Set2', data=df1, ax=axs[0][0])
sns.countplot(x = 'previously_insured', palette = 'Set2', data=df1, ax=axs[0][1])
sns.countplot(x = 'gender', palette = 'Set2', data=df1, ax=axs[0][2])
sns.countplot(x = 'vehicle_age', fpalette = 'Set2', data=df1, ax=axs[1][0])
sns.countplot(x = 'vehicle_damage', palette = 'Set2', data=df1, ax=axs[1][1])

In [ ]:
fig = plt.subplots(figsize = (15,5))
sns.countplot(x='region_code', data=df1)

In [ ]:
policy_1 = df1['policy_sales_channel'].value_counts().iloc[0:55]
policy_1 = policy_1.reset_index()
policy_2 = df1['policy_sales_channel'].value_counts().iloc[55:110]
policy_2 = policy_2.reset_index()
policy_3 = df1['policy_sales_channel'].value_counts().iloc[110:156]
policy_3 = policy_3.reset_index()

fig, axs = plt.subplots(ncols=3, figsize=(15,15))
sns.barplot(y='index', x='policy_sales_channel', data=policy_1, ax=axs[0], orient='h', order=policy_1['index'])
sns.barplot(y='index', x='policy_sales_channel', data=policy_2, ax=axs[1], orient='h', order=policy_2['index'])
sns.barplot(y='index', x='policy_sales_channel', data=policy_3, ax=axs[2], orient='h', order=policy_3['index'])

## 2 - FEATURE ENGINEERING

In [9]:
df2 = df1.copy()

In [10]:
# Fitting vehicle_age column
df2['vehicle_age'] = df2['vehicle_age'].apply(lambda x: 1 if (x == '< 1 Year') else 2 if (x == '1-2 Year') else 3)

# Fitting vehicle_damage column
df2['vehicle_damage'] = df2['vehicle_damage'].apply(lambda x: 1 if (x == 'Yes') else 0)

## 3 - DATA FILTERING

In [11]:
df3 = df2.copy()

## 4 - EXPLORATORY DATA ANALYSIS (EDA)

In [12]:
df4 = df3.copy()

### 4.1 - UNIVARIATE ANALYSIS

In [ ]:
df4.head()

In [ ]:
plt.figure(figsize = (20, 25))
plt.suptitle("Analysis Of Variable Response",fontweight="bold", fontsize=20)

plt.subplot(6,2,1)
sns.countplot(x = 'response', hue = 'gender', palette = 'Set2', data = df4)

plt.subplot(6,2,2)
sns.countplot(x = 'response', hue = 'previously_insured', palette = 'Set2', data = df4)

plt.subplot(6,2,3)
sns.countplot(x = 'response', hue = 'vehicle_age', palette = 'Set2', data = df4)

plt.subplot(6,2,4)
sns.countplot(x = 'response', hue = 'vehicle_damage', palette = 'Set2', data = df4)

plt.subplot(6,2,5)
sns.countplot(x = 'response', hue = 'driving_license', palette = 'Set2', data = df4)

plt.subplot(6,2,6)
sns.countplot(x='response', hue = 'age_group', palette = 'Set2', data=df4)

plt.subplot(6,2,7)
sns.kdeplot(x='age', hue='response', palette = 'Set2', shade=True, data=df4)

plt.subplot(6,2,8)
sns.kdeplot(x='annual_premium', hue='response', palette = 'Set2', shade=True, data=df4)

plt.subplot(6,2,9)
sns.kdeplot(x='day_premium', hue='response', palette = 'Set2', shade=True, data=df4)

plt.subplot(6,2,10)
sns.kdeplot(x='age_premium', hue='response', palette = 'Set2', shade=True, data=df4)

plt.subplot(6,2,11)
sns.kdeplot(x='vintage', hue='response', palette = 'Set2', shade=True, data=df4)

In [ ]:
# 'policy_sales_channel'

# set the figure size
plt.figure(figsize=(25, 10))

# from raw value to percentage
total = df4.groupby('policy_sales_channel')['id'].count().reset_index()
response_1 = df4.loc[df4.response == 1 ].groupby('policy_sales_channel')['id'].count().reset_index()
response_0 = df4.loc[df4.response == 0 ].groupby('policy_sales_channel')['response'].sum().reset_index()
resp = pd.merge(response_1, response_0, how = 'outer', on = 'policy_sales_channel')
resp['id'] = resp['id'].fillna(0)
resp = resp.sort_values(by='policy_sales_channel')
resp['id'] = [i / j * 100 for i,j in zip(resp['id'], total['id'])]
total['id'] = [i / j * 100 for i,j in zip(total['id'], total['id'])]

# bar chart 1 -> top bars (group of 'smoker=No')
bar1 = sns.barplot(x="policy_sales_channel",  y="id", data=total, color='darkblue')

# bar chart 2 -> bottom bars (group of 'smoker=Yes')
bar2 = sns.barplot(x="policy_sales_channel", y="id", data=resp, color='lightblue')

# add legend
plt.xticks(rotation=90)
top_bar = mpatches.Patch(color='darkblue', label='response = No')
bottom_bar = mpatches.Patch(color='lightblue', label='response = Yes')
plt.legend(handles=[top_bar, bottom_bar])

# show the graph
plt.show()

### 4.2 - BIVARIATE ANALYSIS

In [ ]:
#1.The interest on purchase the vehicle insurance is greater for customers that damaged their vehicle before and doesn't have insurance. (FALSE)
d1 = df2[( df2['previously_insured'] == 0) & (df2['vehicle_damage'] == 1)]

ax1 = d1[['response','id']].groupby('response').count().reset_index()
ax1['percentage'] = round(ax1['id'] / d1['id'].count()*100)
ax1

In [ ]:
# 2. The interest on purchase the vehicle insurance is greater for woman than men. (FALSE)
aux2 = pd.crosstab(df2['gender'], df2['response'])
aux2['percentage'] = aux2[1]/(aux2[0]+aux2[1])
aux2

In [ ]:
# 3. The interest on purchase vehicle insurance is greater for vintage customers( 7 months or more )
aux3 = df2[df2['vintage_month'] >= 7 ][['id','response']]
aux4 = df2[df2['vintage_month'] < 7 ][['id','response']]

fix, axs = plt.subplots(ncols = 2, figsize = (15,4))
sns.countplot(x = aux3['response'], ax=axs[0]).set_title('Vintage Customer: 7 months or more')
sns.countplot(x = aux4['response'], ax=axs[1]).set_title('Vintage Customer: 7 months or less')

In [ ]:
aux18 = aux3[['response','id']].groupby('response').count().reset_index()
aux18['percentage'] = round( aux18['id'] / aux3.shape[0] * 100 )
aux18

In [ ]:
aux19 = aux4[['response','id']].groupby('response').count().reset_index()
aux19['percentage'] = round( aux19['id'] / aux4.shape[0] * 100 )
aux19

In [ ]:
# 4. The interest on purchase the vehicle insurance is greater for customers that spend less than 30k for annual premium.
aux5 = df2[df2['annual_premium'] > 30000][['id','response']]
aux6 = df2[df2['annual_premium'] <= 30000][['id','response']]

fix, axs = plt.subplots(ncols = 2, figsize = (15,4))
sns.countplot(x = aux5['response'], ax=axs[0]).set_title('Annual Premium: 30k or more')
sns.countplot(x = aux6['response'], ax=axs[1]).set_title('Annual Premium: 30k or less')

In [ ]:
aux16 = aux5[['response','id']].groupby('response').count().reset_index()
aux16['percentage'] = round( aux16['id'] / aux5.shape[0] * 100 )
aux16

In [ ]:
aux17 = aux6[['response','id']].groupby('response').count().reset_index()
aux17['percentage'] = round( aux17['id'] / aux6.shape[0] * 100 )
aux17

In [ ]:
# 5. The interest on purchase the vehicle insurance is greater for young customers.(Between 18 and 30 years old.)
ax7 = sns.countplot(x = df2['response'], hue=df2['age_group'])

In [ ]:
aux8 = pd.crosstab(df2['age_group'], df2['response'])
aux8['percentage'] = aux8[1]/(aux8[0]+aux8[1])
aux8

In [ ]:
# 6. The interest on purchase the vehicle insurance is greater for customers that have driver license.
aux9 = pd.crosstab(df2['driving_license'], df2['response'])
aux9['percentage'] = round(aux9[1]/(aux9[0]+aux9[1])*100)
aux9

In [ ]:
# 7. The interest on purchase the vehicle insurance is greater for customers that have new cars.
aux10 = pd.crosstab(df2['vehicle_age'], df2['response'])
aux10['percentage'] = round(aux10[1]/(aux10[0]+aux10[1])*100)
aux10

In [ ]:
# 8. The interest on purchase the vehicle insurance is greater for customers that have new cars and have damaged their vehicles.
aux11 = df2[(df2['vehicle_damage'] == 1 )]
aux12 = pd.crosstab(aux11['vehicle_age'], aux11['response'])
aux12['percentage'] = round(aux12[1]/(aux12[0]+aux12[1])*100)
aux12

In [ ]:
# 9. The interest on purchase the vehicle insurance is greater for elderly women.
aux13 = df2[(df2['gender'] == 'Female')]
aux14 = pd.crosstab(aux13['age_group'], aux13['response'])
aux14['percentage'] = round(aux14[1]/(aux14[0]+aux14[1])*100)
aux14

In [ ]:
# 10. The interest on purchase the vehicle insurance is lower for customers that are already insured.
aux15 = pd.crosstab(df2['previously_insured'], df2['response'])
aux15['percentage'] = round(aux15[1]/(aux15[0]+aux15[1])*100)
aux15

**Hypothesis Validation**

1. The interest on purchase the vehicle insurance is greater for customers that damaged their vehicle before and doesn't have insurance.
    **False, of the customers that damaged their car and doesn't have insurance, only 25% show interest in acquire vehicle insurance.**

2. The interest on purchase the vehicle insurance is greater for woman than men.
    **False, only 10% of women show interest in acquire vehicle insurance, whereas 13% of the men show interest on acquire vehicle insurance.**

3. The interest on purchase vehicle insurance is greater for vintage customers ( 7 months or more ).
    **False, the period that customers are on the company doensn't show influency on interest in buying vehicle insurance.**

4. The interest on purchase the vehicle insurance is greater for young customers.(Between 18 and 30 years old.)
    **False, customers that spend more than 30k yearly show greter interest on purchase vehicle insurance.**

5. The interest on purchase the vehicle insurance is greater for young customers.(Between 18 and 30 years old.)
    **False, adults and elderlies show greater interest on buying vehicle insurance.**

6. The interest on purchase the vehicle insurance is greater for customers that have driver license.
    **True, arround 12% of customers that hold a driving license show interest in buying the vehicle insurance.**

7. The interest on purchase the vehicle insurance is greater for customers that have new cars.
    **False, the interest is greater for customers that own an old car.**

8. The interest on purchase the vehicle insurance is greater for customers that have new cars and have damaged their vehicles.
    **False, of the customers who damaged their car, the ones that own a old car show greater interest in buying the vehicle insurance (29%), followed by customers that own used cars (27%).**

9. The interest on purchase the vehicle insurance is greater for elderly women.
    **False, adult women show greater interest in buying the vehicle insurance.**

10. The interest on purchase the vehicle insurance is lower for customers that are already insured.
    **True, less than 1% of customers already insured show interest on purchase the vehicle insurance.**


### 4.3 - MULTIVARIATE ANALYSIS

In [ ]:
correlation = df2.corr().round(2)
plt.figure(figsize = (14,7))
sns.heatmap(correlation, annot = True)

## 5 - DATA PREPARATION

In [13]:
df5 = df4.copy()

### 5.1 - STANDARDIZATION

In [14]:
# Subtrai-se a media e divide-se pelo desvio padrao
ss_ap = pp.StandardScaler()

# annual premium
df5['annual_premium'] = ss_ap.fit_transform( df5[['annual_premium']].values )
#pickle.dump(ss_ap, open('C:/Users/perot/Documents/ds_repos/projects/Health_Insurance_Cross_Sell/src/features/annual_premium_scaler.pkl', 'wb'))

### 5.2 - REESCALING

In [15]:
mms_age = pp.MinMaxScaler()
mms_vintage = pp.MinMaxScaler()

# Age
df5['age'] = mms_age.fit_transform( df5[['age']].values )
#pickle.dump(mms_age, open('C:/Users/perot/Documents/ds_repos/projects/Health_Insurance_Cross_Sell/src/features/age_scaler.pkl', 'wb'))

# Vintage
df5['vintage'] = mms_vintage.fit_transform( df5[['vintage']].values )
#pickle.dump(mms_vintage, open('C:/Users/perot/Documents/ds_repos/projects/Health_Insurance_Cross_Sell/src/features/vintage_scaler.pkl', 'wb'))

### 5.3 - ENCODER

In [16]:
# Frequency encoder for all columns

# gender
fe_gender = df5.groupby('gender').size() / len(df5)
df5.loc[:, 'gender'] = df5['gender'].map( fe_gender )
#pickle.dump(target_encode_gender, open('C:/Users/perot/Documents/ds_repos/projects/Health_Insurance_Cross_Sell/src/features/target_encode_gender_scaler.pkl', 'wb'))

# region_code - Frequency Encoding / Target Encoding / Weighted Targed Encoding
fe_region_code = df5.groupby( 'region_code').size() / len( df5 )
df5.loc[:, 'region_code'] = df5['region_code'].map( fe_region_code )
#pickle.dump(fe_region_code, open('C:/Users/perot/Documents/ds_repos/projects/Health_Insurance_Cross_Sell/src/features/frequency_encode_region_code_scaler.pkl', 'wb'))

# policy_sales_channel - Frequency Encoding / Target Encoding
fe_policy_sales_channel = df5.groupby( 'policy_sales_channel' ).size() / len( df5 ) 
df5.loc[:, 'policy_sales_channel'] = df5['policy_sales_channel'].map( fe_policy_sales_channel )
#pickle.dump(fe_policy_sales_channel, open('C:/Users/perot/Documents/ds_repos/projects/Health_Insurance_Cross_Sell/src/features/frequency_encode_policy_sales_scaler.pkl', 'wb'))

### 5.4 - VALIDATION PREPARATION

#### 5.5.1 -  Apply Feature Engineering on validation dataset

In [17]:
# Fitting vehicle_age column
x_valid['vehicle_age'] = x_valid['vehicle_age'].apply(lambda x: 1 if (x == '< 1 Year') else 2 if (x == '1-2 Year') else 3)

# Fitting vehicle_damage column
x_valid['vehicle_damage'] = x_valid['vehicle_damage'].apply(lambda x: 1 if (x == 'Yes') else 0)

#### 5.5.2 -  Apply Data Transformation on validation dataset

In [18]:
# STANDARDIZATION

# annual premium
x_valid.loc[:, 'annual_premium'] = ss_ap.transform( x_valid[['annual_premium']].values )



# REESCALING

# age
x_valid.loc[:, 'age'] = mms_age.transform( x_valid[['age']].values )

# vintage
x_valid.loc[:, 'vintage'] = mms_vintage.transform( x_valid[['vintage']].values )



# ENCODER

# policy sales channel
x_valid.loc[:, 'policy_sales_channel'] = x_valid['policy_sales_channel'].map( fe_policy_sales_channel )

# region code
x_valid.loc[:, 'region_code'] = x_valid.loc[:, 'region_code'].map( fe_region_code )

# gender
x_valid.loc[:, 'gender'] = x_valid.loc[:, 'gender'].map( fe_gender )

# FILL NAN VALUES
x_valid = x_valid.fillna(0)

## 6 - FEATURE SELECTION

In [19]:
df6 = df5.copy()

In [ ]:
# model definition
forest = en.ExtraTreesClassifier( n_estimators =250, random_state=0, n_jobs=-1 )

# data preparation
df6_n = df6.drop( ['id'], axis=1 )
y_train_n  = y_train.values
forest.fit( df6_n, y_train_n )

In [ ]:
importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0 )
indices = np.argsort(importances)[::-1]

# Print the feature ranking 
print("Feature ranking:")
df = pd.DataFrame()
for i,j in zip( x_train_n, forest.feature_importances_ ):
    aux = pd.DataFrame( {'feature': i, 'importance': j}, index=[0] )
    df = pd.concat( [df,aux], axis = 0 )
    
print( df.sort_values( 'importance', ascending=False ) )

# Plot the impurity-based feature importances of the forest
plt.figure()
plt.title("Feature importances")
plt.bar(range(x_train_n.shape[1]), importances[indices], color="r", yerr=std[indices], align="center")
plt.xticks(range(x_train_n.shape[1]), indices)
plt.xlim([-1, x_train_n.shape[1]])
plt.show()

In [21]:
cols_selected = ['vintage', 'annual_premium','age','region_code','vehicle_damage','policy_sales_channel','previously_insured','vehicle_age']

x_training = df6[ cols_selected ]
x_validation = x_valid[ cols_selected ]
y_validation = y_valid

## 7 - ML MODELS TRAINING

#### 7.1.1 - XGB Model

In [ ]:
# Parameter "scale_pos_weight" definition

from collections import Counter

# ESTIMATE SCALE_POS_WEIGHT
counter = Counter(y_train)
estimate = counter[0]/counter[1]
print('Estimate: %.3f' % estimate)

In [22]:
# model definition
xgb_model = XGBClassifier(scale_pos_weight=7.2)

# model training
xgb_model.fit( x_training, y_train )

# model prediction - Check if the model perform well
yhat_xgb = xgb_model.predict_proba( x_validation )

In [ ]:
fig, axs = plt.subplots(ncols= 3, figsize = (18,5))

# cumulative gain - Metric for sorting problem
skplt.metrics.plot_cumulative_gain(y_validation, yhat_xgb, ax=axs[0],title='Cumulative Gain - XGB');

# Lift Curve
skplt.metrics.plot_lift_curve(y_validation, yhat_xgb,ax=axs[1],title='Lift Curve - XGB');

# Roc Curve
skplt.metrics.plot_roc(y_validation, yhat_xgb, ax=axs[2], title='ROC-Curve - XGB');

plt.tight_layout()

#### 7.1.2 - KNN

In [23]:
# model definition
knn_model = nh.KNeighborsClassifier( n_neighbors=8 )

# model training
knn_model.fit( x_training, y_train )

# model prediction - The generalization POWER
yhat_knn = knn_model.predict_proba( x_validation )

In [ ]:
fig, axs = plt.subplots(ncols= 3, figsize = (18,5))

# cumulative gain - Metric for sorting problem
skplt.metrics.plot_cumulative_gain(y_validation, yhat_knn, ax=axs[0],title='Cumulative Gain - LGBM');

# Lift Curve
skplt.metrics.plot_lift_curve(y_validation, yhat_knn,ax=axs[1],title='Lift Curve - LGBM');

# Roc Curve
skplt.metrics.plot_roc(y_validation, yhat_knn, ax=axs[2], title='ROC-Curve - LGBM');

plt.tight_layout()

#### 7.1.3 - Light gradient Boostin Machine Classifier Model

In [24]:
# Model Definition
lgbm_model = lgbm.LGBMClassifier( learning_rate=0.09,max_depth=-5,random_state=42 )

# Model training
model_lgbm = lgbm_model.fit( x_training, y_train )

# Model Prediction
yhat_lgbm = model_lgbm.predict_proba( x_validation )

In [ ]:
fig, axs = plt.subplots(ncols= 3, figsize = (18,5))

# cumulative gain - Metric for sorting problem
skplt.metrics.plot_cumulative_gain(y_validation, yhat_lgbm, ax=axs[0],title='Cumulative Gain - LGBM');

# Lift Curve
skplt.metrics.plot_lift_curve(y_validation, yhat_lgbm,ax=axs[1],title='Lift Curve - LGBM');

# Roc Curve
skplt.metrics.plot_roc(y_validation, yhat_lgbm, ax=axs[2], title='ROC-Curve - LGBM');

plt.tight_layout()

In [45]:
# XGB treinado e validado com dataset de treinamento
# model definition
xgb_model = XGBClassifier(scale_pos_weight=7.2)
# model training
xgb_model.fit( x_training, y_train )
# model prediction - Check if the model perform well
recall_train_xgb = xgb_model.predict( x_validation )


In [ ]:
#Prerformance
recall_train = mt.recall_score(y_train, recall_train_xgb)
print('Accuracy over Training:{}'.format())

## 8 - MODEL PERFORMANCE

### 8.1 - XGB MODEL PERFORMANCE

In [41]:
from sklearn import metrics as mt

In [40]:
# Copy Data
perf_valid= x_valid.copy()
perf_valid['response'] = y_valid.copy()

# Propensity score
perf_valid['score'] = yhat_xgb[:,1].tolist()

# sorted clients by score
perf_valid = perf_valid.sort_values( 'score', ascending=False )

# Compute precision at K
precision_atK = precision_at_k(perf_valid, k=20000 )

# Compute precision at K
recall_atK = recall_at_k(perf_valid, k=20000 )

xgb_performance = ml_performance('XGB Model', precision_atK, recall_atK )
xgb_performance

,Model Name,Precision_at_K,Recall_at_K
0,XGB Model,0.330833,0.708914


ValueError: Found input variables with inconsistent numbers of samples: [243909, 76222]

### 8.2 - KNN MODEL PERFORMANCE

In [26]:
# Copy Data
perf_valid = x_valid.copy()
perf_valid['response'] = y_valid.copy()

# Propensity score
perf_valid['score'] = yhat_knn[:,1].tolist()

# sorted clients by score
perf_valid = perf_valid.sort_values( 'score', ascending=False )

# Compute precision at K
precision_atK = precision_at_k(perf_valid, k=20000 )

# Compute precision at K
recall_atK = recall_at_k(perf_valid, k=20000 )

KNN_performance = ml_performance('KNN_Model', precision_atK, recall_atK )
KNN_performance

,Model Name,Precision_at_K,Recall_at_K
0,KNN_Model,0.298285,0.639169


### 8.3 - LGBM MODEL PERFORMANCE

In [27]:
# Copy Data
perf_valid = x_valid.copy()
perf_valid['response'] = y_valid.copy()

# Propensity score
perf_valid['score'] = yhat_lgbm[:,1].tolist()

# sorted clients by score
perf_valid = perf_valid.sort_values( 'score', ascending=False )

# Compute precision at K
precision_atK = precision_at_k(perf_valid, k=20000 )

# Compute precision at K
recall_atK = recall_at_k(perf_valid, k=20000 )

lgbm_performance = ml_performance('LGBM Model', precision_atK, recall_atK )
lgbm_performance

,Model Name,Precision_at_K,Recall_at_K
0,LGBM Model,0.333233,0.714056


### 8.3 - PERFORMANCE FOR ALL MODELS

In [28]:
model_performance = pd.concat( [xgb_performance, KNN_performance, lgbm_performance] )
model_performance.sort_values('Precision_at_K', ascending= False)

,Model Name,Precision_at_K,Recall_at_K
0,LGBM Model,0.333233,0.714056
0,XGB Model,0.330833,0.708914
0,KNN_Model,0.298285,0.639169


## 9 - HYPERPARAMETER FINE TUNING

### 9.1 - Cross Validation

In [ ]:
# Concatenation train and validation dataset to use in cross-validation
df_cv = x_train.copy()
df_cv['response'] = y_training

df_aux = x_validation.copy()
df_aux['response'] = y_validation

df_cc = pd.concat([df_cv, df_aux])

In [ ]:
# Dictioary with models instantiated
models = { 'KNN': knn_model,
           'XGB Model': xgb_model,
           'LGBM Model': lgbm_model}

# Cross-validated models performance
model_performance = pd.DataFrame()

for key in models.keys():
    performance_cv = cross_validation(5, key, models[key], df_cc, 20000)
    model_performance = pd.concat([model_performance, performance_cv], axis=1)

model_performance.T

### 9.2 - LGBM Fine Tunning

In [29]:
X = x_training
y = y_train

In [30]:
def objective(trial, X, y):
    param_grid = {
        #"device_type": trial.suggest_categorical("device_type", ['gpu']),
        "n_estimators": trial.suggest_categorical("n_estimators", [10000]),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "num_leaves": trial.suggest_int("num_leaves", 20, 3000, step=20),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 200, 10000, step=100),
        "lambda_l1": trial.suggest_int("lambda_l1", 0, 100, step=5),
        "lambda_l2": trial.suggest_int("lambda_l2", 0, 100, step=5),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0, 15),
        "bagging_fraction": trial.suggest_float(
            "bagging_fraction", 0.2, 0.95, step=0.1
        ),
        "bagging_freq": trial.suggest_categorical("bagging_freq", [1]),
        "feature_fraction": trial.suggest_float(
            "feature_fraction", 0.2, 0.95, step=0.1
        ),
    }

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1121218)

    cv_scores = np.empty(5)
    
    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        model = lgbm.LGBMClassifier(objective="binary", **param_grid)
        model.fit( X_train, y_train, eval_set=[(X_test, y_test)], eval_metric="auc", early_stopping_rounds=100, callbacks=[LightGBMPruningCallback(trial, metric='auc')],  # Add a pruning callback 
                 )
        preds = model.predict_proba(X_test)
        cv_scores[idx] = log_loss(y_test, preds)

    return np.mean(cv_scores)

In [ ]:
study = optuna.create_study(direction='maximize' , study_name="LGBM Classifier")
func = lambda trial: objective(trial, X, y)
study.optimize(func, n_trials=20)

In [ ]:
print(f"\tBest value (rmse): {study.best_value:.5f}")
print(f"\tBest params:")

for key, value in study.best_params.items():
    print(f"\t\t{key}: {value}")

In [31]:
params = {
'n_estimators': 10000,
'learning_rate': 0.14417802056640477,
'num_leaves': 1020,
'max_depth': 4,
'min_data_in_leaf': 9400,
'lambda_l1': 20,
'lambda_l2': 50,
'min_gain_to_split': 11.771936325948504,
'bagging_fraction': 0.2,
'bagging_freq': 1,
'feature_fraction': 0.6000000000000001}

### 9.3 - XGB Fine Tunning

In [ ]:
import random

In [ ]:
# Union of training dataset to select best parameters for XGB Model
x_train_xgb = pd.concat([x_training, y_train], axis=1)

In [ ]:
param = {
     'n_estimators': [1000, 1500, 2000, 2500], 
     'eta': [0.01, 0.03],
     'max_depth': [3, 5, 9],
     'subsample': [0.1, 0.5, 0.7],
     'colsample_bytree': [0.3, 0.7, 0.9],
     'min_child_weight':[3, 8, 15] }

MAX_EVAL = 5

In [ ]:
final_result = pd.DataFrame()

for i in range( MAX_EVAL ):
# choose values for parameters randomly
    hp = { k: random.sample(v, 1)[0] for k, v in param.items() }
    print( hp )
    
#    # model
    model_xgb = XGBClassifier( scale_pos_weight=7.2,
                               colsample_bytree =hp['colsample_bytree'],
                               subsample = hp['subsample'],
                               objective='binary:logistic', 
                               n_estimators=hp['n_estimators'], 
                               max_depth=hp['max_depth'],
                               min_child_weight = hp['min_child_weight'],
                               eta=hp['eta'])
    # performance
    result = cross_validation(2, 'xgb classifier', model_xgb, x_train_xgb, 20000)
    final_result = pd.concat( [final_result, result] )

final_result

###  9.3.2 - Last model training with all datas

#### 9.3.2.1 - Data preparation applied on validation and test datasets.

In [32]:
x_valid = data_preparation( x_valid )

In [33]:
x_test = data_preparation( x_test )

#### 9.3.2.2 - Union of training and validation data.

In [34]:
x_train_treino = pd.concat([ x_training, x_valid ] )
y_train_treino = pd.concat([ y_train, y_valid ] )

#### 9.3.2.3 - Generalization capacity of LGBM Model

In [35]:
# Model Definition
lgbm_model = lgbm.LGBMClassifier(**params )

# Model Training
model_lgbm = lgbm_model.fit( x_train_treino, y_train_treino )

[LightGBM] [Warning] feature_fraction is set=0.6000000000000001, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6000000000000001
[LightGBM] [Warning] min_data_in_leaf is set=9400, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=9400
[LightGBM] [Warning] min_gain_to_split is set=11.771936325948504, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=11.771936325948504
[LightGBM] [Warning] lambda_l1 is set=20, reg_alpha=0.0 will be ignored. Current value: lambda_l1=20
[LightGBM] [Warning] bagging_fraction is set=0.2, subsample=1.0 will be ignored. Current value: bagging_fraction=0.2
[LightGBM] [Warning] lambda_l2 is set=50, reg_lambda=0.0 will be ignored. Current value: lambda_l2=50
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


In [36]:
# Model Prediction
yhat_lgbm = model_lgbm.predict_proba( x_test )

In [37]:
# Copy Data
df_test = x_test.copy()
df_test['response'] = y_test.copy()

# Propensity score
df_test['score'] = yhat_lgbm[:,1].tolist()

# sorted clients by score
df_test = df_test.sort_values( 'score', ascending=False )

# Compute precision at K
precision_atK = precision_at_k(df_test, k=20000 )

# Compute precision at K
recall_atK = recall_at_k(df_test, k=20000 )

lgbm_performance_cv = ml_performance('LGBM Model', precision_atK, recall_atK )
lgbm_performance_cv

,Model Name,Precision_at_K,Recall_at_K
0,LGBM Model,0.305685,0.806065


#### 9.3.2.4 - Generalization capacity of XGB Model

In [38]:
# XGB CROSS VALIDATION

# model definition
model_xgb_cv = XGBClassifier( scale_pos_weight=7.2,
                              colsample_bytree = 0.3,
                              subsample = 0.5,
                              objective='binary:logistic', 
                              n_estimators=1500, 
                              max_depth=9,
                              min_child_weight = 3,
                              eta= 0.03 )

# model training
model_xgb_cv.fit( x_training, y_train )

# model prediction - The generalization POWER
yhat_xgb_cv = model_xgb_cv.predict_proba( x_test )

In [39]:
# Copy Data
df_test = x_test.copy()
df_test['response'] = y_test.copy()

# Propensity score
df_test['score'] = yhat_xgb_cv[:,1].tolist()

# sorted clients by score
df_test = df_test.sort_values( 'score', ascending=False )

# Compute precision at K
precision_atK = precision_at_k(df_test, k=20000 )

# Compute precision at K
recall_atK = recall_at_k(df_test, k=20000 )

xgb_performance_cv = ml_performance('XGB Model', precision_atK, recall_atK )
xgb_performance_cv

,Model Name,Precision_at_K,Recall_at_K
0,XGB Model,0.313234,0.825972


In [ ]:
#pickle.dump(lgbm_model, open('C:/Users/perot/Documents/ds_repos/projects/Health_Insurance_Cross_Sell/src/models/lgbm_model.pkl', 'wb'))

## 10 - DEPLOYING MODEL TO PRODUCTION

### 10.1 - HEALTHINSURANCE CLASS

In [ ]:
import pickle
import inflection
import numpy  as np
import pandas as pd

class HealthInsurance:
    
    def __init__( self ):
        self.home_path                                = ''
        self.frequency_encode_policy_sales_scaler     = pickle.load( open( self.home_path + 'src/features/frequency_encode_policy_sales_scaler.pkl', 'rb') )
        self.frequency_encode_region_code_scaler      = pickle.load( open( self.home_path + 'src/features/frequency_encode_region_code_scaler.pkl', 'rb') )
        self.target_encode_gender_scaler              = pickle.load( open( self.home_path + 'src/features/target_encode_gender_scaler.pkl', 'rb') )
        self.target_encode_vintage_month_scaler       = pickle.load( open( self.home_path + 'src/features/target_encode_vintage_month_scaler.pkl', 'rb') )
        self.age_scaler                               = pickle.load( open( self.home_path + 'src/features/age_scaler.pkl', 'rb') )
        self.avg_vehicle_damage_region_code_scaler    = pickle.load( open( self.home_path + 'src/features/age_scaler.pkl', 'rb') )
        self.avg_vintage_age_scaler                   = pickle.load( open( self.home_path + 'src/features/avg_vintage_age_scaler.pkl', 'rb') )
        self.vintage_scaler                           = pickle.load( open( self.home_path + 'src/features/vintage_scaler.pkl', 'rb') )
        self.age_premium_scaler                       = pickle.load( open( self.home_path + 'src/features/age_premium_scaler.pkl', 'rb') )
        self.annual_premium_scaler                    = pickle.load( open( self.home_path + 'src/features/age_premium_scaler.pkl', 'rb') )
        self.avg_day_premium_policy_scaler            = pickle.load( open( self.home_path + 'src/features/avg_day_premium_policy_scaler.pkl', 'rb') )
        self.day_premium_scaler                       = pickle.load( open( self.home_path + 'src/features/day_premium_scaler.pkl', 'rb') )
        self.median_premium_by_region_scaler          = pickle.load( open( self.home_path + 'src/features/median_premium_by_region_scaler.pkl', 'rb') )
        self.region_premium_scaler                    = pickle.load( open( self.home_path + 'src/features/region_premium_scaler.pkl', 'rb') )
    
    def data_cleaning( self, df5 ):
 
        ## rename Columns
        cols_old = ['id', 'Gender', 'Age', 'Driving_License', 'Region_Code', 'Previously_Insured', 'Vehicle_Age', 'Vehicle_Damage', 'Annual_Premium', 'Policy_Sales_Channel', 'Vintage']

        snakecase = lambda x: inflection.underscore( x )
        cols_new = list( map( snakecase, cols_old ) )
        
        # rename
        df5.columns = cols_new
        
        return( df5 )

    def feature_engineering( self, df5 ):
        
        # Fitting vehicle_age column
        df5['vehicle_age'] = df5['vehicle_age'].apply(lambda x: 1 if (x == '< 1 Year') else 2 if (x == '1-2 Year') else 3)

        # Fitting vehicle_damage column
        df5['vehicle_damage'] = df5['vehicle_damage'].apply(lambda x: 1 if (x == 'Yes') else 0)
        
        # median_premium_by_region
        dict_region_code = df5[['annual_premium', 'region_code']].groupby('region_code').median().to_dict(orient='dict')['annual_premium']
        df5['median_premium_by_region'] = df5['region_code'].map(dict_region_code)

        # moda policy sales chanel por idade
        mode_policy_per_age = df5[['age', 'policy_sales_channel']].groupby('age').agg(pd.Series.mode).to_dict(orient='dict')['policy_sales_channel']
        df5['mode_policy_per_age'] = df5['age'].map(mode_policy_per_age)

        # Media de carros danificados por idade
        avg_carros_danificados_idade = df5[['age', 'vehicle_damage']].groupby('age').mean().to_dict(orient='dict')['vehicle_damage']
        df5['avg_vehicle_damage_per_age'] = df5['age'].map(avg_carros_danificados_idade)

        # Media de carros danificados por regiao
        avg_carros_danificados_regiao = df5[['age', 'region_code']].groupby('age').mean().to_dict(orient='dict')['region_code']
        df5['avg_vehicle_damage_region_code'] = df5['age'].map(avg_carros_danificados_regiao)

        # age_group feature creation
        df5['age_group'] = df5['age'].apply(lambda x: 1 if (x >= 18 | x < 30 ) else 2 if (x >= 30 | x < 60 ) else 3)

        # vintage_month feature cration
        df5['vintage_month'] = round(df5['vintage'] / 31)

        # day_premium feature creation
        df5['day_premium'] = df5['annual_premium']/df5['vintage']

        # age_premium feature cration
        df5['age_premium'] = df5['annual_premium']/df5['age']

        # Calculating Median Annual Premium by Region_code
        premium_rc = df5[['annual_premium', 'region_code']].groupby('region_code').median().to_dict(orient='dict')['annual_premium']
        df5['region_premium'] = df5['region_code'].map(premium_rc)

        # Calculating Median Aday_premium per policy_sales_channel
        day_premium_psc = df5[['day_premium', 'policy_sales_channel']].groupby('policy_sales_channel').mean().to_dict(orient='dict')['day_premium']
        df5['avg_day_premium_policy'] = df5['policy_sales_channel'].map(day_premium_psc)

        # avg vintage per age
        avg_vintage_per_age = df5[['age', 'vintage']].groupby('age').mean().to_dict(orient='dict')['vintage']
        df5['avg_vintage_age'] = df5['age'].map(avg_vintage_per_age)
        
        return( df5 )
    
    def data_preparation( self, df5 ):
              
        # STANDARDIZATION

        # annual premium
        df5['annual_premium'] = self.annual_premium_scaler.transform( df5[['annual_premium']].values )

        # age_premium
        df5['age_premium'] = self.age_premium_scaler.transform( df5[['age_premium']].values )

        # day_premium
        df5['day_premium'] = self.day_premium_scaler.transform( df5[['day_premium']].values )

        # avg_day_premium_policy
        df5['avg_day_premium_policy'] = self.avg_day_premium_policy_scaler.transform( df5[['avg_day_premium_policy']].values )

        # median_premium_by_region
        df5['median_premium_by_region'] = self.median_premium_by_region_scaler.transform( df5[['median_premium_by_region']].values )

        # region_premium
        df5['region_premium'] = self.region_premium_scaler.transform( df5[['region_premium']].values )



        # REESCALING

        # age
        df5['age'] = self.age_scaler.transform( df5[['age']].values )

        # vintage
        df5['vintage'] = self.vintage_scaler.transform( df5[['vintage']].values )

        # avg_vintage_age
        df5['avg_vintage_age'] = self.avg_vintage_age_scaler.transform( df5[['avg_vintage_age']].values )

        # vintage_month
        df5['vintage_month'] = self.vintage_scaler.transform( df5[['vintage_month']].values )

        # avg_vehicle_damage_region_code
        df5['avg_vehicle_damage_region_code'] = self.avg_vehicle_damage_region_code_scaler.transform( df5[['avg_vehicle_damage_region_code']].values )



        # ENCODER

        # policy sales channel
        df5.loc[:, 'policy_sales_channel'] = df5['policy_sales_channel'].map( self.frequency_encode_policy_sales_scaler )

        # region code
        df5.loc[:, 'region_code'] = df5['region_code'].map( self.frequency_encode_region_code_scaler )

        # gender
        df5.loc[:, 'gender'] = df5['gender'].map( self.target_encode_gender_scaler )

        # vintage_month
        df5.loc[:, 'vintage_month'] = df5['vintage_month'].map( self.target_encode_vintage_month_scaler )

        # FILL NAN VALUES
        x_valid = x_valid.fillna(0)
        
        # Feature Selection
        cols_selected = ['vehicle_damage','previously_insured','vintage','day_premium','age_premium','annual_premium','vintage_month','region_code','avg_vehicle_damage_per_age','policy_sales_channel','age','median_premium_by_region','region_premium',
                         'avg_day_premium_policy','vehicle_age','avg_vehicle_damage_region_code','avg_vintage_age','gender']
        
        return df5[cols_selected]
    

    def get_prediction( self, model, original_data, test_data ):
        # prediction
        pred = model.predict_proba( test_data )
        
        # join pred into the original data
        original_data['score'] = pred[:, 1].tolist()
        
        return original_data.to_json( orient='records', date_format='iso' )

### 10.2 - API HANDLER

In [ ]:
import pickle
import pandas as pd
import os
from flask                           import Flask, request, Response
from healthinsurance.healthinsurance import HealthInsurance

# logading model
model = pickle.load( open( 'src/models/xgb_model.pkl', 'rb' ) )
                          
# initialize API
app = Flask( __name__ )

@app.route( '/healthinsurance/predict', methods=['POST'] )
def health_insurance_predict():
    test_json = request.get_json()
    
    if test_json: #there is data
               
        if isinstance( test_json, dict ): # unique example
            test_raw = pd.DataFrame( test_json, index=[0] )
    
        else:
            test_raw = pd.DataFrame( test_json, columns=test_json[0].keys() ) # multiple examples
            
        # Instantiate Rossmann Class
        pipeline = HealthInsurance()

        # data cleaning
        df1 = pipeline.data_cleaning( test_raw )
              
        # feature engineering
        df2 = pipeline.feature_engineering( df1 )

        # Data Preparation
        df3 = pipeline.data_preparation( df2 )
                              
        # prediction
        df_response = pipeline.get_prediction( model, test_raw, df3 )
        
        return df_response
        
    else:
        return Response( '{}', status=200, mimetype='application/json' )

if __name__ == '__main__':
    app.run('0.0.0.0')

#    port = os.environ.get('PORT', 5000)
#    app.run( host='0.0.0.0', port=port )